# Download der Analysedaten

Um unsere Analysen ausführen zu können, benötigen wir einen passenden Datensatz.
Wir wollen dafür die Beiträge und Kommentare aus dem Cyberpunk Subreddit extrahieren.
<hr/>

Als Erstes importieren wir alle packages, die wir im Notebook benötigen.

In [1]:
import sqlite3
import pandas as pd
import requests
from datetime import datetime, timedelta
import time
import json
import praw

Wir definieren uns jetzt noch zwei Funktionen zum Laden von Tabellen in DataFrames und Speichern von DataFrames in
Tabellen.

In [2]:
def pd_read_sql(database, sql):
    with sqlite3.connect(database) as con:
        df = pd.read_sql_query(sql, con)
        return df

def pd_write_sql(database, table, df):
    with sqlite3.connect(database) as con:
        df.to_sql(table, con, index=False, if_exists='replace')

Im Folgenden werden wir die [Pushshift Api](https://pushshift.io/) und Reddit API
[PRAW](https://github.com/praw-dev/praw) benutzen.

Die Pushshift API ermöglicht uns,
Beiträge zu einem gewissen Zeitraum herunterzuladen, was die PRAW API nicht kann. Dafür benutzt die Pushshift API
abgespeicherte Einträge. Das Problem dieser Vorgehensweise ist, dass wenn man Kommentare herunterladen möchte, nur ein
sehr geringer Teil bzw. zu viele Kommentare heruntergeladen werden, da die Speichereinträge veraltet sind.
Zum Beispiel gibt es bei diesem
[Beitrag](https://www.reddit.com/r/cyberpunkgame/comments/kfba3u/megathread_sonyplaystation_will_offer_full/)
laut unserer submissions Tabelle (wird gleich noch erstellt) insgesamt 49,7k Kommentare. Schaut man sich jetzt aber
den Link an, sind es nur noch 37,3k Kommentare (Stand: 20.06.2021), da anscheinend viele gelöscht wurden.


Um genauere Werte zu erhalten, benutzen wir jetzt PRAW, weil diese API direkt über Reddit geht und damit genauere
Ergebnisse liefert (wurde händisch mit zufälligen Beiträgen überprüft).
<hr/>

Jetzt definieren wir uns eine Funktion, die die Top 10 Subreddit Beiträge innerhalb einer Woche im Zeitraum von 10-09-2020 bis 10-03-2021, also drei Monate vor und nach Release, herunterlädt.

Als Vorlage dient dazu die *downloadFromUrl* Funktion von Watchful1
[Github Link zur Funktion](https://github.com/Watchful1/Sketchpad/blob/master/postDownloader.py).
Außerdem machen wir folgende Einschränkungen beim Herunterladen:
 * Keine Beiträge über 18
 * Keine Beiträge, die Spoiler enthalten
 * Keine Beiträge mit Contest Mode

Am Ende werden von den Beiträgen folgende Werte gespeichert:
 * Beitrags Id
 * Autor
 * Titel
 * Text
 * Datum im UTC Format
 * Anzahl Kommentare
 * Ob der Beitrag gesperrt wurde
 * Link zum Beitrag

In [3]:
def downloadSubmissions(subreddit_name):
    topics_dict = {"id": [],
                   "author": [],
                   "title": [],
                   "selftext": [],
                   "created_utc": [],
                   "num_comments": [],
                   "locked": [],
                   "full_link": []
                   }

    url = "https://api.pushshift.io/reddit/search/{}/?size=10&sort=desc&sort_type=num_comments&subreddit={}&before={}" \
          "&after={}&over_18=false&spoiler=false&contest_mode=false"

    date_time_str = '18/03/21 00:00:00'
    start_time = datetime.strptime(date_time_str, '%d/%m/%y %H:%M:%S')

    for i in range(0, 26):
        start_time = start_time - timedelta(days=7)
        end_time = start_time - timedelta(days=7)
        print("START:")
        print(start_time)
        print("END:")
        print(end_time)
        start_epoch = int(start_time.timestamp())
        end_epoch = int(end_time.timestamp())
        new_url = url.format("submission", subreddit_name, str(start_epoch), str(end_epoch))
        json_text = requests.get(new_url, headers={'User-Agent': "Post downloader by /u/Watchful1"})
        time.sleep(1)

        try:
            json_data = json_text.json()
        except json.decoder.JSONDecodeError:
            time.sleep(1)
            continue
        if 'data' not in json_data:
            continue
        objects = json_data['data']
        if len(objects) == 0:
            continue
        for object in objects:
            try:
                topics_dict["id"].append(object["id"])
                topics_dict["author"].append(object["author"])
                topics_dict["title"].append(object["title"])
                topics_dict["selftext"].append(object["selftext"])
                topics_dict["created_utc"].append(datetime.fromtimestamp(object['created_utc']).strftime("%Y-%m-%d"))
                topics_dict["num_comments"].append(object["num_comments"])
                topics_dict["locked"].append(object["locked"])
                topics_dict["full_link"].append(object["full_link"])
            except Exception as err:
                print("Unexpected exception")

    submissions_df = pd.DataFrame(topics_dict)
    pd_write_sql("data.sqlite", "submissions", submissions_df)

Aufruf der Funktion (braucht ein paar Minuten):

In [4]:
downloadSubmissions("cyberpunkgame")

START:
2021-03-11 00:00:00
END:
2021-03-04 00:00:00
START:
2021-03-04 00:00:00
END:
2021-02-25 00:00:00
START:
2021-02-25 00:00:00
END:
2021-02-18 00:00:00
START:
2021-02-18 00:00:00
END:
2021-02-11 00:00:00
START:
2021-02-11 00:00:00
END:
2021-02-04 00:00:00
START:
2021-02-04 00:00:00
END:
2021-01-28 00:00:00
START:
2021-01-28 00:00:00
END:
2021-01-21 00:00:00
START:
2021-01-21 00:00:00
END:
2021-01-14 00:00:00
START:
2021-01-14 00:00:00
END:
2021-01-07 00:00:00
START:
2021-01-07 00:00:00
END:
2020-12-31 00:00:00
START:
2020-12-31 00:00:00
END:
2020-12-24 00:00:00
START:
2020-12-24 00:00:00
END:
2020-12-17 00:00:00
START:
2020-12-17 00:00:00
END:
2020-12-10 00:00:00
START:
2020-12-10 00:00:00
END:
2020-12-03 00:00:00
START:
2020-12-03 00:00:00
END:
2020-11-26 00:00:00
START:
2020-11-26 00:00:00
END:
2020-11-19 00:00:00
START:
2020-11-19 00:00:00
END:
2020-11-12 00:00:00
START:
2020-11-12 00:00:00
END:
2020-11-05 00:00:00
START:
2020-11-05 00:00:00
END:
2020-10-29 00:00:00
START:
2020-

Anzeigen der heruntergeladenen Daten:

In [5]:
df = pd_read_sql("data.sqlite", "SELECT * From submissions")
df.sample(5)

,id,author,title,selftext,created_utc,num_comments,locked,full_link
146,k3ao7y,CL60,Cyberpunk 2077 Release Date in All Timezones.,"*May be different by 1 hour, depending on exac...",2020-11-29,910,0,https://www.reddit.com/r/cyberpunkgame/comment...
216,j7zbt9,CheckingIsMyPriority,People from Game Informer reported citing CDPR...,,2020-10-09,468,0,https://www.reddit.com/r/cyberpunkgame/comment...
141,k3obxq,AutoModerator,Newcomers look here! - FAQ &amp; Simple Questi...,"`Hey Choombas,`\n\nFeel free to use this threa...",2020-11-30,4313,0,https://www.reddit.com/r/cyberpunkgame/comment...
25,lrnxj4,ndim22,Going around the rollercoaster loop on a motor...,,2021-02-24,481,0,https://www.reddit.com/r/cyberpunkgame/comment...
152,jzhflu,Ferninja,Ok all the horny boys on this sub need to chill,Jesus christ its like all I hear about is how ...,2020-11-23,2992,0,https://www.reddit.com/r/cyberpunkgame/comment...


Jetzt möchten wir die passenden Kommentare zu den Beiträgen herunterladen. Dazu definieren wir uns eine neue Funktion.

Von den Kommentaren werden folgende Werte gespeichert:

 * Kommentar Id
 * Autor
 * Text
 * Datum im UTC Format
 * Score
 * Parent Id
 * Link zum Kommentar 
 * Id des zugehörigen Beitrags

In [ ]:
def downloadComments():
    reddit = praw.Reddit(
        #All real values are censored with 'SECRET VALUE'
        client_id="SECRET VALUE",
        client_secret="SECRET VALUE",
        user_agent="SECRET VALUE",
        username="SECRET VALUE",
        password="SECRET VALUE"
    )

    submissions_df = pd_read_sql("submissions.sqlite", "SELECT id FROM submissions")

    topics_dict = {
        "id": [],
        "author": [],
        "body": [],
        "created_utc": [],
        "score": [],
        "parent_id": [],
        "full_link": [],
        "submission_id": []
    }

    i = 0
    for submission_id in submissions_df["id"]:
        time.sleep(2)
        print("Current run: " + str(i))
        i += 1
        submission = reddit.submission(submission_id)
        submission.comments.replace_more(limit=None)
        comments = submission.comments.list()
        for comment in comments:
            topics_dict["id"].append(comment.id)
            topics_dict["author"].append(str(comment.author))
            topics_dict["body"].append(comment.body)
            topics_dict["created_utc"].append(datetime.fromtimestamp(comment.created_utc).strftime("%Y-%m-%d"))
            topics_dict["score"].append(comment.score)
            topics_dict["parent_id"].append(comment.parent_id)
            topics_dict["full_link"].append("https://www.reddit.com" + comment.permalink)
            topics_dict["submission_id"].append(submission_id)

    comments_df = pd.DataFrame(topics_dict)
    pd_write_sql("data.sqlite", "comments", comments_df)

Aufruf der Funktion (dauert einen halben Tag. Um das Ergebnis zu sehen, bitte zur nächsten Zelle springen):

In [ ]:
downloadComments()

Anzeigen der heruntergeladenen Daten:

In [7]:
df = pd_read_sql("data.sqlite", "SELECT * From comments")
df.sample(5)

,id,author,body,created_utc,score,parent_id,full_link,submission_id
335733,gf8y80i,poopf1nger,Hey there anyone with a 2080 super want to cha...,2020-12-10,2,t3_k0llf4,https://www.reddit.com/r/cyberpunkgame/comment...,k0llf4
378845,gahqzoi,OedipusNo,"I'm pretty sure, I can't remember the source, ...",2020-10-29,3,t1_gahjpc0,https://www.reddit.com/r/cyberpunkgame/comment...,jk6mv5
326374,gfanmx0,heydaddio69,Thanks. Sorry.,2020-12-10,2,t1_gfankj1,https://www.reddit.com/r/cyberpunkgame/comment...,k3obxq
351167,gcvbzkq,Sermokala,And even in that world its competing with Go y...,2020-11-19,1,t1_gcuk5v2,https://www.reddit.com/r/cyberpunkgame/comment...,jx72zf
321395,gf3pkdz,deswesher,And are they lots of bugs and glitches like re...,2020-12-08,2,t1_gf3p8c5,https://www.reddit.com/r/cyberpunkgame/comment...,k2r53r


## Quellen ##

 * https://pushshift.io/
 * https://github.com/praw-dev/praw
 * https://www.reddit.com/r/cyberpunkgame/comments/kfba3u/megathread_sonyplaystation_will_offer_full/